In [7]:
import pymongo
from pymongo import MongoClient, UpdateOne
import json

# Adding for query to find top ten stations 
#from pymongo.collection import Collection
#from pymongo.aggregation import Aggregation

In [8]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [9]:
# check our list of databases
print(mongo.list_database_names())

['admin', 'chicago_bikes', 'classDB', 'config', 'fruits_db', 'gardenDB', 'local', 'met', 'travel_db', 'uk_food']


In [10]:
# assign the database to a variable name
db = mongo.chicago_bikes

In [11]:
# review the collections in our new database
print(db.list_collection_names())

['divvy_rides_by_season', 'withLatLong', 'withoutStationName', 'Top10EndStations', 'weather_daily', 'Top10StartStations', 'withStationName', 'divvy_ridedata_merged', 'divvy_ridedata', 'RouteDistance', 'Top10Routes', 'divvy_rides_by_month']


In [12]:
pipeline = [
    {
        "$match": {
            "$and": [
                { "start_lat": {"$or": [{ "$ne": "" }, {"$ne": 0}]}},
                { "start_lng": {"$or": [{ "$ne": "" }, {"$ne": 0}]}},
                { "end_lat": {"$or": [{ "$ne": "" }, {"$ne": 0}]}},
                { "end_lng": {"$or": [{ "$ne": "" }, {"$ne": 0}]}}
            ]
        }
    },
    {
        "$addFields": {
            "start_lat": { "$toDouble": "$start_lat" },
            "start_lng": { "$toDouble": "$start_lng" },
            "end_lat": { "$toDouble": "$end_lat" },
            "end_lng": { "$toDouble": "$end_lng" }
        }
    },
    {
        "$addFields": {
            "distance": {
                "$sqrt": {
                    "$add": [
                        {
                            "$pow": [
                                { "$subtract": ["$end_lat", "$start_lat"] },
                                2
                            ]
                        },
                        {
                            "$pow": [
                                {
                                    "$multiply": [
                                        { "$subtract": ["$end_lng", "$start_lng"] },
                                        { "$cos": { "$avg": ["$start_lat", "$end_lat"] } }
                                    ]
                                },
                                2
                            ]
                        }
                    ]
                }
            }
        }
    },
    {
        "$sort": {"distance": -1}
    }
    {"$out": "RouteDistance"}
]

# Perform the aggregation
result = list(withLatLong.aggregate(pipeline))

# Assign to a variable
RouteDistance = db['RouteDistance']

NameError: name 'withLatLong' is not defined

In [ ]:
# review the collections in our new database
print(db.list_collection_names())